#Exportar modelo TensorFlow.keras para poder utilizar en TensorFlow.js
Fuentes:

https://www.tensorflow.org/guide/keras/serialization_and_saving

https://medium.com/@sebastingarcaacosta/how-to-export-a-tensorflow-2-x-keras-model-to-a-frozen-and-optimized-graph-39740846d9eb


 https://www.tensorflow.org/js/tutorials/conversion/import_keras


In [1]:
#@title Instalar parquete TensorFlow.js
#@markdown (porque es necesario para exportar el modelo entrenado)
!pip install tensorflowjs


INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 59.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-d

In [2]:
#@title Acceder al Drive

# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)



Mounted at /content/gdrive


In [3]:
#@title Generar FrozenGraph del Modelo.keras
#@markdown (no es posible realizar la conversión de .keras o .H5 porque luego no funciona en TF.js)

import os.path
import shutil
from tensorflow import keras
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np


path_modelo = '/content/gdrive/My Drive/IA/demoModelDeployment/modelo'  #@param {type:"string"}

#  carga el modelo a congelar
model = load_model(path_modelo+"/model.keras")
print("\nModelo cargado de ", path_modelo,"\n")

#path of the directory where you want to save your model
frozen_path = path_modelo+ '/frozen'
# name of the .pb file
frozen_graph_filename = 'frozen_graph'
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))
# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 60)
print("-Frozen model layers: ")
for layer in layers:
    print(layer)
print("-" * 60)
print("-Frozen model inputs: ")
print(frozen_func.inputs)
print("-Frozen model outputs: ")
print(frozen_func.outputs)
print("-" * 60)

# Save frozen graph to disk
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_path,
                  name=f"{frozen_graph_filename}.pb",
                  as_text=False)
# Save its text representation
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_path,
                  name=f"{frozen_graph_filename}.pbtxt",
                  as_text=True)

print("\n\nFrozenGraphs grabado en ", frozen_path,"\n")



Modelo cargado de  /content/gdrive/My Drive/IA/demoModelDeployment/modelo 

------------------------------------------------------------
Frozen model layers: 
x
RNA_1/hidd_1_1/Cast/ReadVariableOp/resource
RNA_1/hidd_1_1/Add/ReadVariableOp/resource
RNA_1/output_1/Cast/ReadVariableOp/resource
RNA_1/output_1/Add/ReadVariableOp/resource
RNA_1/hidd_1_1/Cast/ReadVariableOp
RNA_1/hidd_1_1/MatMul
RNA_1/hidd_1_1/Add/ReadVariableOp
RNA_1/hidd_1_1/Add
RNA_1/output_1/Cast/ReadVariableOp
RNA_1/output_1/MatMul
RNA_1/output_1/Add/ReadVariableOp
RNA_1/output_1/Add
RNA_1/output_1/Softmax
NoOp
Identity
------------------------------------------------------------
-Frozen model inputs: 
[<tf.Tensor 'x:0' shape=(None, 4) dtype=float32>]
-Frozen model outputs: 
[<tf.Tensor 'Identity:0' shape=(None, 4) dtype=float32>]
------------------------------------------------------------


FrozenGraphs grabado en  /content/gdrive/My Drive/IA/demoModelDeployment/modelo/frozen 



In [4]:
#@title Convertir FrozenGraph para TensorFlow.js
import os.path
import shutil

#@markdown Indicar nombre del nodo de salida de la red para poder hacer la conversión
#@markdown (en este caso es una capa softmax)
output_node_name = 'RNA_1/output_1/Softmax'  #@param {type:"string"}

# copia frozenGraphs a disco local
shutil.copytree(frozen_path, '/content', dirs_exist_ok=True)
path_modeloFrozenFile = "/content/frozen_graph.pb"
if os.path.exists(path_modeloFrozenFile):

  path_modeloJS = "/content/modeloJS"

  # realiza la conversión a tfjs_graph_model
  ## https://github.com/tensorflow/tfjs/tree/master/tfjs-converter#javascript-to-python
  !tensorflowjs_converter --output_node_names='{output_node_name}' --input_format tf_frozen_model --output_format tfjs_graph_model {path_modeloFrozenFile} {path_modeloJS}

  if os.path.exists(path_modeloJS):
    print("\nModelo exportado para TF.js en ", path_modeloJS,"\n")

    path_modeloJS_drive = path_modelo + '/modeloJS'

    # copia archivos generados al drive
    shutil.copytree(path_modeloJS, path_modeloJS_drive, dirs_exist_ok=True)
    drive.mount('/content/gdrive', force_remount=True)
    if os.path.exists(path_modeloJS_drive):
      print("\nArchivos de modelos TF.js movidos a ", path_modeloJS_drive)
      print("-- usar tf.loadGraphModel() para cargarlo --\n")

else:
  print("\nNo se encuentra FrozenGraphs en ", path_modeloFrozenFile,"\n")



2024-08-06 13:17:33.182857: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 13:17:33.305299: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 13:17:33.305389: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-06 13:17:36.215346: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

Modelo exportado para TF.js en  /content/modeloJS 

Mounted at /content/gdrive

Archivos de modelos TF.js movidos a  /content/gdrive/My Drive/IA/demoModelDeployment/modelo/modeloJS
-- usar tf.loadGraphModel() para cargarlo --



In [6]:
#@title Instalar kora para obtener Ids de archivos
#@markdown ( volver a ejecutar si tira error )

!pip install kora
print("\n")
from kora.xattr import get_id


In [7]:
#@title Obtener IDs de archivos del Modelo TF.js en el Drive


for x in [["FileId_modelJS", "model.json"], ["FileId_modelWeights", "group1-shard1of1.bin"]]:
  var = x[0]
  fn = path_modeloJS_drive + "/" + x[1]
  print("\n//- ", fn, ":")
  print("const " + var + " = \"" + get_id(fn) + "\"")



//-  /content/gdrive/My Drive/IA/demoModelDeployment/modelo/modeloJS/model.json :
const FileId_modelJS = "1-X-JTfpXw1007YJAEP33SoIJe6CGLGOq"

//-  /content/gdrive/My Drive/IA/demoModelDeployment/modelo/modeloJS/group1-shard1of1.bin :
const FileId_modelWeights = "1-OA6Qs2iyQGbTtbpVzri-2Kwy1-ZCTdj"


In [8]:
#@title Obtener información complemetaria para el Modelo

import joblib

fn_scaler = path_modelo+"/scaler.joblib"
if os.path.isfile(fn_scaler):
  print("\n// * Se detecta que el modelo tiene definido un Scaler para normalizar los datos, pero no es posible exportarlo para usar en JavaScript! \n")


fn_clases = path_modelo+"/CLASES.txt"
CLASES = []
if os.path.isfile(fn_clases):
  with open(fn_clases, 'r') as f:
    # carga datos
    auxData = f.readlines()
  for c in auxData:
    CLASES.append( c.replace("\n", "") )
  print("\n// * CLASES definidas cargado de ", fn_clases, ":")
  print("const Model_CLASES = ", CLASES, "\n")
else:
  print("\n//* CLASES no encontradas en ", fn_clases, "\n")
  print("const Model_CLASES = [] \n")

print("")


// * CLASES definidas cargado de  /content/gdrive/My Drive/IA/demoModelDeployment/modelo/CLASES.txt :
const Model_CLASES =  ['na', 'Setosa', 'Versicolor', 'Virginica'] 


